In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import env
from scipy import stats
import csv



In [3]:
def get_db_url(db):
    return f'mysql+pymysql://{env.user}:{env.password}@{env.host}/{db}'

    
def get_data_from_mysql():
    query = '''
    Select props.`id` as property_id, 
    props.`bathroomcnt` as bathroom_count,
    props.`bedroomcnt` as bedroom_count,
    props.`calculatedfinishedsquarefeet` as calc_finish_sq_ft,
    props.`taxvaluedollarcnt` as assessed_property_value,
    props.`taxamount` as tax_paid,
    (props.`taxamount` / props.`taxvaluedollarcnt`) * 100 as tax_rate,
    props.`fips` as county_code,
    svi.`COUNTY` as county
    from zillow.properties_2017 as props
        JOIN zillow.predictions_2017 as pred
        on zillow.props.id = zillow.pred.id
    JOIN svi_db.`svi2016_us_county` as svi
    on zillow.props.fips = svi_db.svi.fips
    Where transactiondate like "2017-05%%" or transactiondate like "2017-06%%" 
    and props.`propertylandusetypeid` not in (31, 246, 247, 248)
    AND
    props.`calculatedfinishedsquarefeet` IS NOT NULL
    AND 
    props.`bathroomcnt` != 0
    AND
    props.`bedroomcnt` != 0

    '''

    df = pd.read_sql(query, get_db_url('zillow'))
    return df    
   


,property_id,bathroom_count,bedroom_count,calc_finish_sq_ft,assessed_property_value,tax_paid,tax_rate,county_code,county
0,1772,2.0,4.0,1604.0,498347.0,6089.82,1.222004,6037.0,Los Angeles
1,2028,3.0,3.0,2384.0,549917.0,6679.55,1.214647,6037.0,Los Angeles
2,3273,2.0,3.0,1574.0,235272.0,3876.31,1.647587,6037.0,Los Angeles
3,3429,2.0,2.0,1619.0,340000.0,4206.15,1.237103,6037.0,Los Angeles
4,3908,3.0,3.0,1936.0,418146.0,5278.74,1.262416,6037.0,Los Angeles


In [5]:
def clean_data(df):
    df = df.dropna()
    df = df[df.bathroom_count > 0]
    df = df[df.bedroom_count > 0]
    return df

In [6]:
def wrangle_zillow():
    df = get_data_from_mysql()
    df = clean_data(df)
    return df


,customer_id,monthly_charges,total_charges,tenure
0,0013-SMEOE,109.70,7904.25,71
1,0014-BMAQU,84.65,5377.80,63
2,0016-QLJIS,90.45,5957.90,65
3,0017-DINOC,45.20,2460.55,54
4,0017-IUDMW,116.80,8456.75,72
5,0019-EFAEP,101.30,7261.25,72
6,0019-GFNTW,45.05,2560.10,56
7,0020-INWCK,95.75,6849.40,71
8,0040-HALCW,20.40,1090.60,54
9,0042-RLHYP,19.70,1396.90,69


,property_id,bathroom_count,bedroom_count,calc_finish_sq_ft,assessed_property_value,tax_paid,tax_rate,county_code,county
0,1772,2.0,4.0,1604.0,498347.0,6089.82,1.222004,6037.0,Los Angeles
1,2028,3.0,3.0,2384.0,549917.0,6679.55,1.214647,6037.0,Los Angeles
2,3273,2.0,3.0,1574.0,235272.0,3876.31,1.647587,6037.0,Los Angeles
3,3429,2.0,2.0,1619.0,340000.0,4206.15,1.237103,6037.0,Los Angeles
4,3908,3.0,3.0,1936.0,418146.0,5278.74,1.262416,6037.0,Los Angeles
5,4505,3.0,2.0,2408.0,2017254.0,24353.42,1.207256,6037.0,Los Angeles
7,5444,3.0,6.0,2610.0,381514.0,5708.90,1.496380,6037.0,Los Angeles
8,5931,2.0,4.0,1721.0,629951.0,6861.51,1.089213,6037.0,Los Angeles
9,6926,1.0,3.0,1136.0,305481.0,5488.23,1.796586,6037.0,Los Angeles
10,7018,1.0,2.0,1088.0,79930.0,1498.86,1.875216,6037.0,Los Angeles
